In [1]:
import pandas as pd
import numpy as np
import fastparquet

In [2]:
df = pd.read_parquet('meow_df.parquet', engine='fastparquet')

In [3]:
df.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id,level_0,bid_price1,ask_price1,bid_price2,...,seconds_in_bucket_before,bid_price1_before,ask_price1_before,bid_price2_before,ask_price2_before,bid_size1_before,ask_size1_before,bid_size2_before,ask_size2_before,stock_id_before
0,5,21,1.002301,326,12,97,13,1.001422,1.002818,1.001370,...,19,1.001422,1.002301,1.001370,1.002405,3,126,2,100,97
1,5,46,1.002778,128,4,97,17,1.002818,1.003232,1.002301,...,44,1.001732,1.002818,1.001422,1.002922,83,28,3,100,97
2,5,50,1.002818,55,1,97,20,1.002353,1.003025,1.002301,...,48,1.002818,1.003232,1.002353,1.003749,55,101,103,100,97
3,5,57,1.003155,121,5,97,23,1.002508,1.003646,1.002457,...,55,1.002353,1.003025,1.002301,1.003181,3,20,100,100,97
4,5,68,1.003646,4,1,97,27,1.002870,1.003749,1.002818,...,63,1.002818,1.003646,1.002767,1.003749,11,4,14,200,97


In [4]:
df['stock_id'].unique().tolist()

[97, 98, 99, -8070346077764945415, 5, 8, 48]

In [5]:
df_tr = pd.read_parquet('optiver/trade_train.parquet', engine='fastparquet')
df_tr.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id
0,5,21,1.002301,326,12,0
1,5,46,1.002778,128,4,0
2,5,50,1.002818,55,1,0
3,5,57,1.003155,121,5,0
4,5,68,1.003646,4,1,0


In [6]:
df_tr['stock_id'].unique().tolist()

[0,
 1,
 10,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 108,
 109,
 11,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 118,
 119,
 120,
 122,
 123,
 124,
 125,
 126,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 2,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 29,
 3,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 4,
 40,
 41,
 42,
 43,
 44,
 46,
 47,
 48,
 5,
 50,
 51,
 52,
 53,
 55,
 56,
 58,
 59,
 6,
 60,
 61,
 62,
 63,
 64,
 66,
 67,
 68,
 69,
 7,
 70,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 8,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 9,
 90,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [7]:
len(df)

38382716

In [8]:
len(df_tr)

38382741